In [0]:
# !pip install textblob vaderSentiment scikit-learn

  Obtaining dependency information for textblob from https://files.pythonhosted.org/packages/02/07/5fd2945356dd839974d3a25de8a142dc37293c21315729a41e775b5f3569/textblob-0.18.0.post0-py3-none-any.whl.metadata
  Obtaining dependency information for vaderSentiment from https://files.pythonhosted.org/packages/76/fc/310e16254683c1ed35eeb97386986d6c00bc29df17ce280aed64d55537e9/vaderSentiment-3.3.2-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/626.3 kB ? eta -:--:--
   ╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/626.3 kB ? eta -:--:--
   ━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/626.3 kB 793.3 kB/s eta 0:00:01
   ━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/626.3 kB 551.5 kB/s eta 0:00:02
   ━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/626.3 kB 716.3 kB/s eta 0:00:01
   ━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.6/626.3 kB 635.1 kB/s eta 0:00:01
   ━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.6/626.3 kB 938.6 kB/s eta 0:00:01
   ━━━━━━━━━━━━━━━╺━━━━━━━━

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from textblob import TextBlob
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml import Pipeline

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Reddit Sentiment & Emotion Analysis") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

### Define UDFs for Sentiment and Emotion Analysis
Next, define functions for sentiment polarity (using TextBlob) and emotion detection based on sentiment polarity.

In [0]:
# Define UDFs for sentiment and emotion analysis
def get_sentiment(text):
    if text:
        blob = TextBlob(text)
        return blob.sentiment.polarity
    return 0.0  # Return 0.0 for empty or null texts

def get_emotion(text):
    sentiment = get_sentiment(text)
    if sentiment > 0.1:
        return "Positive"
    elif sentiment < -0.1:
        return "Negative"
    else:
        return "Neutral"

# Register UDFs for sentiment and emotion analysis
spark.udf.register("get_sentiment", get_sentiment)
spark.udf.register("get_emotion", get_emotion)

<function __main__.get_emotion(text)>

### Load Data from Silver Layer

In [0]:
# Load data from the Silver layer (assuming it is stored in Delta format)
silver_df = spark.read.format("delta").table("big_data_analytics_v.big_data_analytics_sesssion_v.silver_reddit_posts")

### Apply Sentiment and Emotion Analy sis  on 'title' and 'description'
This part involves applying the sentiment and emotion analysis functions (get_sentiment and get_emotion) to the title and description columns of the Reddit posts.

In [0]:
# Perform sentiment and emotion analysis on 'title' and 'description'
transformed_df = silver_df.withColumn("title_polarity", F.expr("get_sentiment(title)")) \
                          .withColumn("title_emotion", F.expr("get_emotion(title)")) \
                          .withColumn("description_polarity", F.expr("get_sentiment(description)")) \
                          .withColumn("description_emotion", F.expr("get_emotion(description)"))

### TF-IDF Feature Extraction
This part involves applying the sentiment and emotion analysis functions (get_sentiment and get_emotion) to the title and description columns of the Reddit posts.

In [0]:
# Replace null values in 'title' and 'description' columns with empty strings
transformed_df = transformed_df.fillna({'title': '', 'description': ''})

# Perform TF-IDF Feature Extraction for 'title' and 'description'
tokenizer_title = Tokenizer(inputCol="title", outputCol="title_words")
tokenizer_description = Tokenizer(inputCol="description", outputCol="description_words")

hashing_tf_title = HashingTF(inputCol="title_words", outputCol="title_tfidf")
hashing_tf_description = HashingTF(inputCol="description_words", outputCol="description_tfidf")

idf_title = IDF(inputCol="title_tfidf", outputCol="title_tfidf_features")
idf_description = IDF(inputCol="description_tfidf", outputCol="description_tfidf_features")

# Create a pipeline for TF-IDF feature extraction
pipeline = Pipeline(stages=[tokenizer_title, tokenizer_description, hashing_tf_title, 
                            hashing_tf_description, idf_title, idf_description])

# Fit and transform the data to extract features
pipeline_model = pipeline.fit(transformed_df)
final_df = pipeline_model.transform(transformed_df)

# Display the final DataFrame
display(final_df)

Uploading artifacts:   0%|          | 0/4 [00:00<?, ?it/s]

post_id,title,description,subreddit,author,score,created_at,url,title_polarity,title_emotion,description_polarity,description_emotion,title_words,description_words,title_tfidf,description_tfidf,title_tfidf_features,description_tfidf_features
1h8jsw4,What has Major to do with Aviation ?,"Koi sharam hoti hey, koi haya hoti hey",pakistan,DanishJaved,566,2024-12-07T03:43:29Z,https://i.redd.it/56xdl83elc5e1.jpeg,0.0625,Neutral,0.0,Neutral,"List(what, has, major, to, do, with, aviation, ?)","List(koi, sharam, hoti, hey,, koi, haya, hoti, hey)","Map(vectorType -> sparse, length -> 262144, indices -> List(27576, 32869, 81566, 115994, 126466, 132786, 133107, 151058), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))","Map(vectorType -> sparse, length -> 262144, indices -> List(36821, 61756, 92093, 158962, 159989, 193464), values -> List(1.0, 1.0, 2.0, 2.0, 1.0, 1.0))","Map(vectorType -> sparse, length -> 262144, indices -> List(27576, 32869, 81566, 115994, 126466, 132786, 133107, 151058), values -> List(1.5686159179138452, 3.1780538303479458, 2.2617630984737906, 2.4849066497880004, 2.772588722239781, 2.4849066497880004, 3.1780538303479458, 2.772588722239781))","Map(vectorType -> sparse, length -> 262144, indices -> List(36821, 61756, 92093, 158962, 159989, 193464), values -> List(3.1780538303479458, 2.772588722239781, 6.3561076606958915, 6.3561076606958915, 3.1780538303479458, 3.1780538303479458))"
1h8kuhq,Reality of Pakistan Stock Market,,pakistan,Nixture24,294,2024-12-07T04:44:02Z,https://v.redd.it/zl16ene5wc5e1,0.0,Neutral,0.0,Neutral,"List(reality, of, pakistan, stock, market)",List(),"Map(vectorType -> sparse, length -> 262144, indices -> List(8145, 18697, 51576, 97454, 219087), values -> List(1.0, 1.0, 1.0, 1.0, 1.0))","Map(vectorType -> sparse, length -> 262144, indices -> List(249180), values -> List(1.0))","Map(vectorType -> sparse, length -> 262144, indices -> List(8145, 18697, 51576, 97454, 219087), values -> List(3.1780538303479458, 3.1780538303479458, 3.1780538303479458, 1.4733057381095203, 1.4733057381095203))","Map(vectorType -> sparse, length -> 262144, indices -> List(249180), values -> List(0.31585294941847725))"
1h8pj8y,"مولوی طاہر اشرفی کے والد کی ""ايمان افروز"" تحریر",ہمارے مولویوں نے کوئی کسر نہیں چھوڑی,pakistan,desolatoration,92,2024-12-07T10:12:26Z,https://i.redd.it/8s6wtecsie5e1.jpeg,0.0,Neutral,0.0,Neutral,"List(مولوی, طاہر, اشرفی, کے, والد, کی, ""ايمان, افروز"", تحریر)","List(ہمارے, مولویوں, نے, کوئی, کسر, نہیں, چھوڑی)","Map(vectorType -> sparse, length -> 262144, indices -> List(34365, 92443, 96533, 106356, 133991, 168014, 248527, 249626, 260523), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))","Map(vectorType -> sparse, length -> 262144, indices -> List(12316, 51476, 60101, 60711, 67217, 173925, 254370), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))","Map(vectorType -> sparse, length -> 262144, indices -> List(34365, 92443, 96533, 106356, 133991, 168014, 248527, 249626, 260523), values -> List(3.1780538303479458, 3.1780538303479458, 3.1780538303479458, 3.1780538303479458, 3.1780538303479458, 3.1780538303479458, 3.1780538303479458, 3.1780538303479458, 3.1780538303479458))","Map(vectorType -> sparse, length -> 262144, indices -> List(12316, 51476, 60101, 60711, 67217, 173925, 254370), values -> List(3.1780538303479458, 3.1780538303479458, 3.1780538303479458, 3.1780538303479458, 3.1780538303479458, 3.1780538303479458, 3.1780538303479458))"
1h8r4sf,Pakistan Air Force to Field Chinese J-35 Fifth Generation Fighters Before 2027 - Reports,"I have seen other few reports of Pakistan increasing its naval fleet, strengthening it's navy etc. Im not against this, pakistan is completely in its right to improve its defence. But I thought pakistan is having an economic crisis at the moment, how is it affording to do it? Or am I just out of loop and it's normal. I haven't seen something like this anywhere else.",pakistan,weebmaster696,54,2024-12-07T12:07:57Z,https://militarywatchmagazine.com/articl

### Topic Modeling
Topic modeling can help discover the main themes in your title and description text. We will use Latent Dirichlet Allocation (LDA) for topic modeling.

In [0]:
from pyspark.ml.clustering import LDA
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline

# If 'combined_features' already exists, remove it
if 'combined_features' in final_df.columns:
    final_df = final_df.drop('combined_features')

# Combine title and description TF-IDF features
assembler = VectorAssembler(
    inputCols=["title_tfidf_features", "description_tfidf_features"],
    outputCol="combined_features"
)
final_df = assembler.transform(final_df)

# Apply LDA for topic modeling (k=5 topics with default settings)
lda = LDA(
    k=2,  # Set number of topics
    maxIter=1,  # Set number of iterations
    featuresCol="combined_features",  # Use combined features
    topicDistributionCol="topicDistribution"  # Assign topic distributions
)
lda_model = lda.fit(final_df)
topics_df = lda_model.transform(final_df)

# Show the top topics and associated data
display(topics_df.select("topicDistribution", "title", "description"))

topicDistribution,title,description
"Map(vectorType -> dense, length -> 2, values -> List(0.942437969557363, 0.05756203044263703))",What has Major to do with Aviation ?,"Koi sharam hoti hey, koi haya hoti hey"
"Map(vectorType -> dense, length -> 2, values -> List(0.9365000033020255, 0.06349999669797453))",Reality of Pakistan Stock Market,
"Map(vectorType -> dense, length -> 2, values -> List(0.02735413443636068, 0.9726458655636394))","مولوی طاہر اشرفی کے والد کی ""ايمان افروز"" تحریر",ہمارے مولویوں نے کوئی کسر نہیں چھوڑی
"Map(vectorType -> dense, length -> 2, values -> List(0.01161275829008726, 0.9883872417099128))",Pakistan Air Force to Field Chinese J-35 Fifth Generation Fighters Before 2027 - Reports,"I have seen other few reports of Pakistan increasing its naval fleet, strengthening it's navy etc. Im not against this, pakistan is completely in its right to improve its defence. But I thought pakistan is having an economic crisis at the moment, how is it affording to do it? Or am I just out of loop and it's normal. I haven't seen something like this anywhere else."
"Map(vectorType -> dense, length -> 2, values -> List(0.08271783154087572, 0.9172821684591242))",Protest against AJK ordinance intensifies,
"Map(vectorType -> dense, length -> 2, values -> List(0.9483695943421115, 0.05163040565788851))",BBC News report on British Mirpuris,
"Map(vectorType -> dense, length -> 2, values -> List(0.04480150435931044, 0.9551984956406895))","Early morning snaps from Model town park, Lahore",
"Map(vectorType -> dense, length -> 2, values -> List(0.9235194111522739, 0.07648058884772614))",Peshawar near to SUNSET ☀️,
"Map(vectorType -> dense, length -> 2, values -> List(0.05558366613235218, 0.9444163338676478))",Even Snakes can't survive Punjab polss,
"Map(vectorType -> dense, length -> 2, values -> List(0.9771885220328391, 0.022811477967160963))",Beautiful kid on Islamabad streets selling flowers 🥺😭,Please try to help them ( ik we should not entertain beggars ) but 30-100rps won't do you any harm.


### Sentiment Trends Visualization

In [0]:
# Extract date or month for trend analysis
final_df = final_df.withColumn("date", F.to_date(final_df["created_at"]))

# Optionally, if you want to analyze by month
final_df = final_df.withColumn("month", F.date_format(final_df["created_at"], "yyyy-MM"))

# Optionally, if you want to analyze by day of the week
final_df = final_df.withColumn("day_of_week", F.dayofweek(final_df["created_at"]))

# Group by date or month and calculate average sentiment polarity
sentiment_trends_df = final_df.groupBy("month").agg(
    F.avg("title_polarity").alias("avg_title_polarity"),
    F.avg("description_polarity").alias("avg_description_polarity")
)

# Display the aggregated sentiment data
sentiment_trends_df.show()

+-------+-------------------+------------------------+
|  month| avg_title_polarity|avg_description_polarity|
+-------+-------------------+------------------------+
|2024-12|0.03953900709219858|    0.004020307399161646|
+-------+-------------------+------------------------+



In [0]:
import plotly.express as px

# Assuming sentiment_trends_df is already calculated as shown in your code above
sentiment_trends_pandas = sentiment_trends_df.toPandas()

# Create an interactive plot for sentiment trends
fig = px.line(sentiment_trends_pandas, x='month', y=['avg_title_polarity', 'avg_description_polarity'],
              labels={'month': 'Month', 'value': 'Average Sentiment Polarity'},
              title='Sentiment Trends Over Time')

# Add markers for better visualization
fig.update_traces(mode='lines+markers')

# Customize layout for better appearance
fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Average Sentiment Polarity",
    title="Sentiment Trends Over Time",
    template="plotly_dark",  # Optional: Use a dark theme
    autosize=True
)

# Show the interactive plot
fig.show()

Exception ignored in: <function JavaWrapper.__del__ at 0x7f94d796fc40>
Traceback (most recent call last):
  File "/databricks/spark/python/pyspark/ml/wrapper.py", line 53, in __del__
    if SparkContext._active_spark_context and self._java_obj is not None:
                                              ^^^^^^^^^^^^^^
AttributeError: 'LDA' object has no attribute '_java_obj'


### Save the Final Data

In [0]:
from pyspark.sql.functions import col

# Flatten the STRUCT columns and select only the 'values' field from 'title_tfidf_features' and 'description_tfidf_features'
final_df_transformed = final_df.select(
    "post_id",
    "title_polarity",
    "title_emotion",
    "description_polarity",
    "description_emotion",
    col("title_tfidf_features.values").alias("title_tfidf_values"),
    col("description_tfidf_features.values").alias("description_tfidf_values"),
    "features",
    "combined_features"
)

# Create a temporary view of the transformed DataFrame
final_df_transformed.createOrReplaceTempView("ml_reddit_posts_temp")

# Insert data into the table using Spark SQL
spark.sql("""
    INSERT INTO big_data_analytics_v.big_data_analytics_sesssion_v.ml_reddit_posts
    SELECT 
        post_id,
        title_polarity,
        title_emotion,
        description_polarity,
        description_emotion,
        title_tfidf_values,
        description_tfidf_values,
        features,
        combined_features
    FROM ml_reddit_posts_temp
""")

# Log the completion of the insert process
print("ML-related data inserted into the table successfully!")

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-3056731227740120>, line 4
      1 from pyspark.sql.functions import col
      3 # Flatten the STRUCT columns and select only the 'values' field from 'title_tfidf_features' and 'description_tfidf_features'
----> 4 final_df_transformed = final_df.select(
      5     "post_id",
      6     "title_polarity",
      7     "title_emotion",
      8     "description_polarity",
      9     "description_emotion",
     10     col("title_tfidf_features.values").alias("title_tfidf_values"),
     11     col("description_tfidf_features.values").alias("description_tfidf_values"),
     12     "features",
     13     "combined_features"
     14 )
     16 # Create a temporary view of the transformed DataFrame
     17 final_df_transformed.createOrReplaceTempView("ml_reddit_posts_temp")

File /databricks/spark/python/pyspark/instrumentation_uti